In [7]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform


baseline_ecps = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")

In [15]:
import pandas as pd

period = 2026

# Household-level variables
has_esi_hh = baseline_ecps.calculate("has_esi", map_to="household", period=period)
hh_emp_inc = baseline_ecps.calculate("employment_income", map_to="household", period=period)
age_hh = baseline_ecps.calculate("age_head", map_to="household", period=period)
household_size = baseline_ecps.calculate("household_size", map_to="household", period=period)

# Weights
hh_wt = hh_emp_inc.weights

# Force boolean
has_esi_bool = (has_esi_hh == 1)

# Criteria: has ESI and $0 employment income
mask = has_esi_bool & (hh_emp_inc == 0)

# Build DataFrame
df = pd.DataFrame({
"household_id": hh_emp_inc.index,
"has_esi": has_esi_bool,
"employment_income": hh_emp_inc,
"weight": hh_wt,
"age_head": age_hh,
"household_size": household_size,
})

df_sel = df.loc[mask].copy()

# Sort by weight descending
df_sorted = df_sel.sort_values("weight", ascending=False)

# Totals
total_weighted_hh = df_sel["weight"].sum()
total_unweighted_hh = len(df_sel)

print(df_sorted.head(10))  # top 10 households by weight
print("\nTotals:")
print(f"  Weighted households: {total_weighted_hh:,.0f}")
print(f"  Unweighted households: {total_unweighted_hh:,}")


       household_id  has_esi  employment_income         weight  age_head  \
7101           7101     True                0.0  509672.531250      71.0   
11956         11956     True                0.0  310583.531250      77.0   
2531           2531     True                0.0  198853.843750      56.0   
11818         11818     True                0.0  186070.500000      50.0   
718             718     True                0.0  147077.000000      69.0   
7324           7324     True                0.0  146454.421875      69.0   
5893           5893     True                0.0  141963.656250      85.0   
608             608     True                0.0  136438.953125      60.0   
8736           8736     True                0.0  135514.218750      37.0   
6265           6265     True                0.0  127626.453125      77.0   

       household_size  
7101                1  
11956               1  
2531                1  
11818               2  
718                 1  
7324               